In [ ]:
%matplotlib notebook

from experimental.beacon_sim import plot_trials
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import functools

In [ ]:
files = {
    'baseline': '/tmp/beacon_sim_log/baseline.pb',
    'baseline_100xBearingNoise_5xRangeNoise': '/tmp/beacon_sim_log/baseline_100xBearingNoise_5xRangeNoise.pb',
    'baseline_100xBearingNoise_5xRangeNoise_100xHeadingProcessPerRtM': '/tmp/beacon_sim_log/baseline_100xBearingNoise_5xRangeNoise_100xHeadingProcessPerRtM.pb',
    'new': '/tmp/results.pb'
}

results_by_label = {}
covs_with_idx_by_label = {}
for label, fname in files.items():
    results_by_label[label] = plot_trials.load_results(fname)
    covs = plot_trials.compute_covariance_size(results_by_label[label])
    covs_with_idx = [(i, x) for i, x in enumerate(covs)]
    covs_with_idx_by_label[label] = sorted(covs_with_idx, key=lambda x: x[1])

In [ ]:
plt.figure(figsize=(10, 6))
for label, covs in covs_with_idx_by_label.items():
    frac = np.linspace(0, 1, len(covs))
    plt.plot([x for _, x in covs], range(len(covs)), label=label)
plt.xlabel('|$\Sigma$|')
plt.ylabel('Fraction')
plt.legend()

In [ ]:
def missing_from_configuration(configuration, world_map):
    idxs = []
    for i in range(len(world_map.fixed_beacons.beacons)):
        if configuration & (1 << i):
            idxs.append(world_map.fixed_beacons.beacons[i].id)
    return sorted(idxs)

def plot_environment(road_map, world_map, plan, local_from_start, goal, configuration=None):
    if configuration is None:
        configuration = 0
    missing_beacons = missing_from_configuration(configuration, world_map)
        
    # plot the road map
    # Plot the edges
    line_segments = []
    for r in range(road_map.adj.num_rows):
        for c in range(r+1, road_map.adj.num_cols):
            idx = r * road_map.adj.num_cols + c
            if (road_map.adj.data[idx] == 1):
                line_segments.append([tuple(road_map.points[r].data), tuple(road_map.points[c].data)])
    edges = mpl.collections.LineCollection(line_segments, colors=(0.8, 0.8, 0.6, 1.0))
    ax = plt.gca()
    ax.add_collection(edges)
    
    # Plot the plan
    plan_points = []
    for node_idx in plan:
        if node_idx == -1:
            plan_points.append(tuple(local_from_start.translation.data))
        elif node_idx == -2:
            plan_points.append((goal.x, goal.y))
        else:
            plan_points.append(tuple(road_map.points[node_idx].data))
    plan_x = [pt[0] for pt in plan_points]
    plan_y = [pt[1] for pt in plan_points]
    plt.plot(plan_x, plan_y, 'b')
    
    # Plot the start
    plt.plot(*local_from_start.translation.data, 'md', markersize=10)
    
    # Plot the goal
    plt.plot(goal.x, goal.y, 'g*', markersize=15)
    
    # Plot the nodes
    rm_x = [pt.data[0] for pt in road_map.points]
    rm_y = [pt.data[1] for pt in road_map.points]
    plt.plot(rm_x, rm_y, 'rs')
    
    TEXT_X_OFFSET = 0.2
    TEXT_Y_OFFSET = 0.2
#     for i, pt in enumerate(road_map.points):
#         plt.text(pt.data[0] + TEXT_X_OFFSET, pt.data[1] + TEXT_Y_OFFSET, i)
        
    # Plot the beacons
    beacon_xs = [beacon.pos_x_m for beacon in world_map.fixed_beacons.beacons if beacon.id not in missing_beacons]
    beacon_ys = [beacon.pos_y_m for beacon in world_map.fixed_beacons.beacons if beacon.id not in missing_beacons]
    beacon_ids = [beacon.id for beacon in world_map.fixed_beacons.beacons if beacon.id not in missing_beacons]
    
    for x, y, beacon_id in zip(beacon_xs, beacon_ys, beacon_ids):
        plt.text(x + TEXT_X_OFFSET, y + TEXT_Y_OFFSET, beacon_id)
    
    plt.plot(beacon_xs, beacon_ys, 'b^')
    plt.axis('equal')
    

In [ ]:
plt.figure(figsize=(10,6))
label = 'new'
results = results_by_label[label]
covs_with_idx = covs_with_idx_by_label[label]
cov_idx = 12
plot_environment(results.road_map, results.world_map_config, results.plan, results.local_from_start, results.goal, configuration=covs_with_idx[cov_idx][0])